#### **Hydrodynamic Characterization and Virtual Pressure Sensing of an Oscillating Water Column (OWC)**
#### **Purpose:** This project implements a **Linear Virtual Pressure Sensor (VPS)** to estimate the internal pneumatic chamber pressure of a fixed OWC wave energy converter. By processing experimental tank-test data from the **Marinet 2** "Round Robin" campaign (Lir National Ocean Test Facility), the study characterizes the device's hydrodynamic response and develops a fault-tolerant soft pressure sensor to support real-time control and grid integration.

##### **Data Source:** [Marinet 2 Fixed Oscillating Water Column Wave Energy Converter Test Data Set – UCC](https://www.seanoe.org/data/00697/80895/) (Lyden et al., 2021).
##### **Author:** Bello Oluwatobi
##### **Last Updated:** September 25, 2025

### #1 Installing Libraries

In [ ]:
# install necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [ ]:
# %% [markdown]
# ## 1. Load Processed Data
# We load the datasets pre-processed in MATLAB.

### #2 Loading the datasets

In [ ]:
# loading the training data (operational sea states - Hs between 0.025m and 0.125m)
df_train = pd.read_csv('../Processed_Data_Files/owc_training_data.csv')

# loading the regular/normal wave test data(Hs = 0.025m)
df_norm = pd.read_csv('../Processed_Data_Files/owc_normal_test_data.csv')

# loading the highly irregular wave test data (Hs = 0.158m)
df_irr = pd.read_csv('../Processed_Data_Files/owc_high_irregular_test_data.csv')

In [ ]:
print(f"Training Wave Data Loaded: {df_train.shape[0]} samples")
print(f"Irregular Wave Data Loaded: {df_irr.shape[0]} samples")
print(f"Normal Wave Data Loaded: {df_norm.shape[0]} samples")

### #3 Data Exploration

In [ ]:
# setting the plotting style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams.update({'font.size': 12, 'font.family': 'serif'})

In [ ]:
# setting the sampling frequency (100 Hz as per the original data collection)
fs=100

#### #3.1 Plotting the time-domain response of the different datasets

In [ ]:
# defining the datasets, their time windows and titles for the subplots
datasets = [
    (df_train, 'Operational Sea State (Training Data)', 2000, 3000),
    (df_irr, 'Highly Irregular Sea State ($H_s=0.158m$)', 2000, 3000),
    (df_norm, 'Normal Sea State (Regular Wave)', 500, 1000)
]

# creating a figure with 3 subplots stacked vertically
fig, axes = plt.subplots(3, 1, figsize=(12, 12), sharex=False)

# adding the main title for the plots
fig.suptitle('Time-Domain Response of Incident Surface Elevation and OWC Chamber Pneumatic Pressure', 
             fontsize=16, fontweight='bold')

# looping through each dataset and plotting on its corresponding axis
for i, (df, title, start, end) in enumerate(datasets):
    ax = axes[i]
    
    # selecting the specific time window
    window = slice(start, end)
    t = df['Time'][window]
    
    # normalizing the data using the Z-Scores
    wave_norm = (df['WG1'][window] - df['WG1'].mean()) / df['WG1'].std()
    pres_norm = (df['P_Chamber'][window] - df['P_Chamber'].mean()) / df['P_Chamber'].std()
    
    # plotting the normalized wave and pressure data
    ax.plot(t, wave_norm, label='Incident Wave (WG1)', color='#1f77b4', linewidth=2)
    ax.plot(t, pres_norm, label='OWC Chamber Pressure ($P_{ch}$)', color='#d62728', linestyle='--', linewidth=2)
    
    # adding titles and styling the subplot
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_ylabel(r'Normalized Amplitude ($\sigma$)', fontsize=12)
    ax.grid(True, linestyle=':', alpha=0.6)
    
    # adding the legend to the first plot
    if i == 0:
        ax.legend(loc='upper right', frameon=True)

# setting the common x-axis label at the bottom
axes[-1].set_xlabel('Time (s)', fontsize=12)

plt.tight_layout()

# adjusting the top of the subplots
fig.subplots_adjust(top=0.92)

# saving the figure
plt.savefig('../results/time_domain_response_all_datasets.png', dpi=300, bbox_inches='tight')
plt.show()

#### #3.2 Plotting the hydrodynamic phase lags

In [ ]:
# defining the datasets and titles
datasets = [
    (df_train, 'Operational Sea State (Training Data)'),
    (df_irr, 'Highly Irregular Sea State ($H_s=0.158m$)'),
    (df_norm, 'Normal Sea State (Regular Wave)')
]

# creating the figure with 3 subplots
fig, axes = plt.subplots(3, 1, figsize=(10, 15), sharex=True)

# seting the main title
fig.suptitle('Cross-Correlation Analysis: Determination of Hydrodynamic Phase Lag', 
             fontsize=16, fontweight='bold')

# looping through each dataset
for i, (df, title) in enumerate(datasets):
    ax = axes[i]
    
    # calculating the cross-correlation
    corr = signal.correlate(df['P_Chamber'] - df['P_Chamber'].mean(), 
                            df['WG1'] - df['WG1'].mean(), mode='full')
    lags = signal.correlation_lags(len(df['P_Chamber']), len(df['WG1']), mode='full')
    lag_time = lags / fs

    # 2. normalizing the correlation coefficient (R_xy)
    corr /= np.max(np.abs(corr))

    # masking window for relevant +/- 5 seconds
    mask = (lag_time > -5) & (lag_time < 5)
    ax.plot(lag_time[mask], corr[mask], color='black', linewidth=1.5)

    # finding and annotating the peak
    peak_idx = np.argmax(corr)
    peak_lag = lag_time[peak_idx]
    
    ax.axvline(peak_lag, color='red', linestyle='--', 
               label=f'Hydrodynamic Phase Lag: {peak_lag:.2f} s')

    # adding titles and y labels
    ax.set_title(title, fontsize=12)
    ax.set_ylabel(r'Correlation ($R_{xy}$)', fontsize=10)
    ax.grid(True, linestyle=':', alpha=0.6)
    ax.legend(loc='upper right')

# setting a general x-axis label for all subplots
axes[-1].set_xlabel('Time Lag ($s$)', fontsize=12)


plt.tight_layout()
fig.subplots_adjust(top=0.92)

# saving the figure
plt.savefig('../results/cross_correlation_lag_all_datasets.png', dpi=300, bbox_inches='tight')
plt.show()

#### #3.3 Plotting OWC internal free surface vs Chamber pressure

In [ ]:
# defining the datasets, their titles and colors
datasets = [
    (df_train, 'Operational Sea State (Training Data)', 'navy'),
    (df_irr, 'Highly Irregular Sea State ($H_s=0.158m$)', 'purple'),
    (df_norm, 'Normal Sea State (Regular Wave)', 'darkgreen')
]

# creating the figure with 3 subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True)

# setting the main title
fig.suptitle('Hysteresis Analysis: OWC Internal Free Surface vs. Chamber Pressure', 
             fontsize=16, fontweight='bold')

# looping through the datasets
for i, (df, title, color) in enumerate(datasets):
    ax = axes[i]
    
    # plotting the scatter points
    ax.scatter(df['WG6'], df['P_Chamber'], 
               alpha=0.1, s=5, c=color, label='Experimental Data')

    # adding titles and labels
    ax.set_title(title, fontsize=12)
    ax.set_xlabel('Internal Wave Gauge WG6 (m)', fontsize=10)
    ax.grid(True, linestyle=':', alpha=0.6)
    
    # adding a zero line for reference
    ax.axhline(0, color='black', linewidth=0.8, alpha=0.5)
    ax.axvline(df['WG6'].mean(), color='black', linewidth=0.8, alpha=0.5)

# setting y-label only on the first plot
axes[0].set_ylabel('Chamber Pressure (Pa)', fontsize=12)

plt.tight_layout()
fig.subplots_adjust(top=0.85)

# saving the figure
plt.savefig('../results/hysteresis_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

#### #3.4 Plotting Power Spectral Density

In [ ]:
# defining the datasets and their titles
datasets = [
    (df_train, 'Operational Sea State (Training Data)'),
    (df_irr, 'Highly Irregular Sea State ($H_s=0.158m$)'),
    (df_norm, 'Normal Sea State (Regular Wave)')
]

# creating figure with 3 subplots
fig, axes = plt.subplots(3, 1, figsize=(10, 15), sharex=True)

# setting the main title
fig.suptitle('Power Spectral Density Comparison: Incident Wave Energy vs. Pneumatic Response', 
             fontsize=16, fontweight='bold')

for i, (df, title) in enumerate(datasets):
    ax = axes[i]
    
    # calculating the power spectral density (PSD) using Welch's method
    f_w, Pxx_wave = signal.welch(df['WG1'].values, fs, nperseg=1024)
    f_p, Pxx_pres = signal.welch(df['P_Chamber'].values, fs, nperseg=1024)
    
    # plotting using semilogy for better dynamic range visualization
    ax.semilogy(f_w, Pxx_wave, label='Incident Wave Energy', color='#1f77b4', linewidth=1.5)
    ax.semilogy(f_p, Pxx_pres, label='Pneumatic Response Energy', color='#d62728', linewidth=1.5)
    
    # adding title and y-label
    ax.set_title(title, fontsize=12)
    ax.set_ylabel(r'Energy Density ($V^2/Hz$)', fontsize=10)
    ax.set_xlim(0, 2.0) # focusing on the dominant wave frequency range (0-2 Hz)
    ax.grid(True, which='both', linestyle=':', alpha=0.5)
    
    if i == 0:
        ax.legend(loc='upper right')

# setting the common x-axis label
axes[-1].set_xlabel('Frequency (Hz)', fontsize=12)


plt.tight_layout()
fig.subplots_adjust(top=0.92)

# saving the figure
plt.savefig('../results/PSD_energy_spectrum.png', dpi=300, bbox_inches='tight')
plt.show()

### #4 Feature Engineering

In [ ]:
# defining time history for the training data
# i.e. how far back into the past to consider 
# (0s, 0.05s, 0.10s ... up to 0.50s) for WG1 
lags = [0, 5, 10, 20, 30, 40, 50]

# looping through the lags and creating new columns
for lag in lags:
    col_name = f'WG1_t-{lag}'
    df_train[col_name] = df_train['WG1'].shift(lag)


# inspecting the first rows of the training data with the new lag features
df_train.head()

In [ ]:
# checking the shape
df_train.shape

In [ ]:
# checking for missing values
df_train.isna().sum()

In [ ]:
# dropping the empty rows created by shifting (NaNs)
df_train.dropna(inplace=True)

In [ ]:
# checking the new shape of the data
df_train.shape

In [ ]:
# inspecting the data
df_train.head()

In [ ]:
# defining time history for the highly irregular wave test data
for lag in lags:
    col_name = f'WG1_t-{lag}'
    df_irr[col_name] = df_irr['WG1'].shift(lag)

df_irr.dropna(inplace=True)

### #5 Data preprocessing

In [ ]:
# defining the input features (X) and target variable (y)
feature_cols = [f'WG1_t-{lag}' for lag in lags]

# for the training data
X = df_train[feature_cols]
y = df_train['P_Chamber']

# for the highly irregular wave test data (to be used for evaluation)
X_irr = df_irr[feature_cols]
y_irr = df_irr['P_Chamber']

In [ ]:
# splitting the training data into 80/20 for training and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
# initializing the Scaler
scaler = StandardScaler()

# fitting the scaler on the training data only (to avoid data leakage)
X_train_scaled = scaler.fit_transform(X_train)

# transforming the validation and highy irregular wave data using the scaler
X_val_scaled = scaler.transform(X_val)
X_irr_scaled = scaler.transform(X_irr)

### #6 Model Training

In [ ]:
# training the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

In [ ]:
# training the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, max_depth=15, n_jobs=-1, random_state=42)
rf_model.fit(X_train_scaled, y_train)

### #7 Model Evaluation

In [ ]:
# model evaluation on the operational sea state data
y_pred_lr = lr_model.predict(X_val_scaled)
y_pred_rf = rf_model.predict(X_val_scaled)

In [ ]:
# calculate the r2 scores
r2_lr = r2_score(y_val, y_pred_lr)
r2_rf = r2_score(y_val, y_pred_rf)

#calculating the RMSE for both models
rmse_lr = np.sqrt(mean_squared_error(y_val, y_pred_lr))
rmse_rf = np.sqrt(mean_squared_error(y_val, y_pred_rf))

print("Validation Results on Operational Sea State")
print(f"Linear Regression R2: {r2_lr:.3f} (RMSE: {rmse_lr:.2f} Pa)")
print(f"Random Forest     R2: {r2_rf:.3f} (RMSE: {rmse_rf:.2f} Pa)")

In [ ]:
# model evaluation on the highly irregular sea state data
irr_pred_lr = lr_model.predict(X_irr_scaled)
irr_pred_rf = rf_model.predict(X_irr_scaled)

# calculating the r2 scores
irr_r2_lr = r2_score(y_irr, irr_pred_lr)
irr_r2_rf = r2_score(y_irr, irr_pred_rf)

#calculating the RMSE for both models
rmse_lr = np.sqrt(mean_squared_error(y_irr, irr_pred_lr))
rmse_rf = np.sqrt(mean_squared_error(y_irr, irr_pred_rf))
    
print("Validation Results on Highly Irregular Sea State (Unseen Data)")
print(f"Linear Regression R2: {irr_r2_lr:.3f}, RMSE: {rmse_lr:.2f} Pa")
print(f"Random Forest     R2: {irr_r2_rf:.3f}, RMSE: {rmse_rf:.2f} Pa")

### #8 Visualization of Model Performance

In [ ]:
# plotting the model performance on the highly irregular wave data

# selecting a 20-second window for detailed visualization
start = 2000
end = 4000
time_axis = df_irr['Time'].iloc[start:end]

plt.figure(figsize=(15, 6))

# plotting the actual sensor data (black)
plt.plot(time_axis, y_irr.iloc[start:end], color='black', label='Actual Sensor (Ground Truth)', alpha=0.7)

# plotting the Linear Regression model prediction (red-dashed)
plt.plot(time_axis, irr_pred_lr[start:end], color='red', linestyle='--', label='Virtual Pressure Sensor (Linear Regression Model)')

plt.title(f'Model Performance on Highly Irregular Sea State Data ($H_s=0.158m$)')
plt.xlabel('Time (s)')
plt.ylabel('OWC Chamber Pressure (Pa)')
plt.legend()
plt.tight_layout()

# Save the figure for the report
plt.savefig('../results/model_performance_high_irregular_wave.png', dpi=300)
plt.show()

In [ ]:
# plotting the linearity and correlation between actual and predicted pressures
plt.figure(figsize=(8, 8))

# plotting the scatter points for actual vs predicted pressures 
plt.scatter(y_irr, irr_pred_lr, alpha=0.1, color='blue', s=10)

# drawing the ideal fit line
min_val = min(y_irr.min(), irr_pred_lr.min())
max_val = max(y_irr.max(), irr_pred_lr.max())
plt.plot([min_val, max_val], [min_val, max_val], 'k--', linewidth=2, label='Ideal Fit')

plt.title('Linearity and Correlation Analysis: Actual vs. Estimated Pneumatic Pressure')
plt.xlabel('Actual Pressure (Pa)')
plt.ylabel('Predicted Pressure (Pa)')
plt.legend(loc='upper left') # Explicitly set legend to upper left
plt.grid(True)

# adding the model performance metrics as a text box on the plot
textstr = '\n'.join((
    r'$R^2=%.3f$' % (irr_r2_lr, ),
    r'$MAE=%.2f$ Pa' % (rmse_lr, )))

# Define the style of the text box
props = dict(boxstyle='round', facecolor='white', alpha=0.9, edgecolor='gray')

# placing the text box in the lower right corner
plt.gca().text(0.65, 0.15, textstr, transform=plt.gca().transAxes, fontsize=12,
        verticalalignment='top', bbox=props)

plt.tight_layout()

plt.savefig('../results/linearity_correlation_analysis.png', dpi=300)
plt.show()

### #9 Saving the Model

In [ ]:
# saving the linear regression model and scaler
joblib.dump(lr_model, '../model_assets/owc_virtual_sensor_model.pkl')
joblib.dump(scaler, '../model_assets/scaler.pkl')